# ECSE556 - HW1

## Preprocessing data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")
import os

np.random.seed(0)  # fixing the seed for reproducibility

In [ ]:
data_path = os.path.join("data", "gdsc_expr_postCB.csv")
data = pd.read_csv(data_path, index_col=0, header=0)
data.head()

In [ ]:
data.shape

We transpose the dataframe because the standard is to have the columns be the features and the rows the samples.

In [ ]:
df = data.transpose()
df.head()

In [ ]:
df.shape

In [ ]:
# checking missing values
df.isnull().sum().sum()

We split the dataset into training and evaluation, to avoid data leakage and biases. (seed is fixed)

In [ ]:
# split the dataset
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=0)
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)
print(f"Shape of training set: {train.shape}")
print(f"Shape of test set: {test.shape}")

We scale the dataset. Most algorithm work best when the data are scaled.

In [ ]:
from sklearn.preprocessing import StandardScaler

features_scaler = StandardScaler()
np_train_scaled = features_scaler.fit_transform(df_train)
df_train_scaled = pd.DataFrame(
    np_train_scaled, index=df_train.index, columns=df_train.columns
)

## Dimensionality Reduction

### PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
np_pca_projected = pca.fit_transform(df_train_scaled)

In [ ]:
np_pca_projected.shape

In [ ]:
sns.scatterplot(x=np_pca_projected[:, 0], y=np_pca_projected[:, 1])
plt.title("PCA dimensionality reduction")

### t-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, perplexity=50, n_iter=2000, random_state=0)
np_tsne_projected = tsne.fit_transform(df_train_scaled)

In [ ]:
np_tsne_projected.shape

In [ ]:
sns.scatterplot(x=np_tsne_projected[:, 0], y=np_tsne_projected[:, 1])
plt.title("t-SNE dimensionality reduction")

### UMAP

In [ ]:
from umap import UMAP

In [ ]:
np_umap_projected = UMAP().fit_transform(df_train_scaled)

In [ ]:
np_umap_projected.shape

In [ ]:
sns.scatterplot(x=np_umap_projected[:, 0], y=np_umap_projected[:, 1])
plt.title("UMAP dimensionality reduction")

### All together

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sns.scatterplot(x=np_pca_projected[:, 0], y=np_pca_projected[:, 1], ax=axes[0])
axes[0].set_title("PCA")

sns.scatterplot(x=np_umap_projected[:, 0], y=np_umap_projected[:, 1], ax=axes[1])
axes[1].set_title("UMAP")

sns.scatterplot(x=np_tsne_projected[:, 0], y=np_tsne_projected[:, 1], ax=axes[2])
axes[2].set_title("t-SNE")

We observe that the three methods produce very different representations of the input features.

## Clustering

### Agglomerative and K-means clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
agglo = AgglomerativeClustering(n_clusters=3)

In [ ]:
agglo_clustering = agglo.fit_predict(df_train_scaled)

In [ ]:
df_agglo_clustering = pd.DataFrame(agglo_clustering, index=df_train_scaled.index)
df_agglo_clustering.head()

In [ ]:
agglo_clustering.shape

In [ ]:
np.unique(agglo_clustering)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=3)

In [ ]:
k_clustering = kmeans.fit_predict(df_train_scaled)

In [ ]:
df_k_clustering = pd.DataFrame(k_clustering, index=df_train_scaled.index)
df_k_clustering.head()

In [ ]:
k_clustering.shape

In [ ]:
np.unique(k_clustering)

Counting the number of samples attributed to each cluster

In [ ]:
print("For agglomerative clustering:")
for i in range(3):
    print(f"Number of samples in cluster {i}: {np.sum(agglo_clustering == i)}")
print("For K-means clustering:")
for i in range(3):
    print(f"Number of samples in cluster {i}: {np.sum(k_clustering == i)}")

We note that the proportion of samples in each cluster is very close. For further comparison of the methods, we should see if the samples belong to the same clusters. This is what we do in the following section.

In [ ]:
# plot the two distributions of classes on each subplot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].set_title("Agglomerative Clustering")
axes[1].set_title("K-Means Clustering")
# hist plot
sns.histplot(agglo_clustering, ax=axes[0], discrete=True, stat="percent", shrink=0.9)
axes[0].xaxis.set_ticks([0, 1, 2], ["Cluster-0", "Cluster-1", "Cluster-2"])
sns.histplot(k_clustering, ax=axes[1], discrete=True, stat="percent", shrink=0.9)
axes[1].xaxis.set_ticks([0, 1, 2], ["Cluster-0", "Cluster-1", "Cluster-2"])

### Visualizing the clusters on the 2D-projections from part 1

Clusters from the agglomerative method visualized on the 2D-projections.

In [ ]:
# Get the dictionary of Seaborn palettes
seaborn_palettes = sns.palettes.SEABORN_PALETTES

# Get the list of palette names
palette_names = list(seaborn_palettes.keys())

# Print the list of palette names
print(palette_names)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sns.scatterplot(x=np_pca_projected[:, 0], y=np_pca_projected[:, 1], ax=axes[0], hue=agglo_clustering, palette="muted6")
axes[0].set_title("PCA")

sns.scatterplot(x=np_umap_projected[:, 0], y=np_umap_projected[:, 1], ax=axes[1], hue=agglo_clustering, palette="muted6")
axes[1].set_title("UMAP")

sns.scatterplot(x=np_tsne_projected[:, 0], y=np_tsne_projected[:, 1], ax=axes[2], hue=agglo_clustering, palette="muted6")
axes[2].set_title("t-SNE")

Clusters from the k-means clustering method visualized on the 2D-projections from part 1.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sns.scatterplot(x=np_pca_projected[:, 0], y=np_pca_projected[:, 1], ax=axes[0], hue=k_clustering, palette="muted6")
axes[0].set_title("PCA")

sns.scatterplot(x=np_umap_projected[:, 0], y=np_umap_projected[:, 1], ax=axes[1], hue=k_clustering, palette="muted6")
axes[1].set_title("UMAP")

sns.scatterplot(x=np_tsne_projected[:, 0], y=np_tsne_projected[:, 1], ax=axes[2], hue=k_clustering, palette="muted6")
axes[2].set_title("t-SNE")

### Jacard similarity scores between clusters

In [ ]:
from sklearn.metrics import jaccard_score

In [ ]:
similarity_scores = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        similarity_score = jaccard_score(
            (agglo_clustering == i).astype("int32"), (k_clustering == j).astype("int32")
        )
        similarity_scores[i, j] = similarity_score
        print(f"Similarity score for clusters {i} vs {j}: {similarity_score}")

In [ ]:
df_similarity_scores = pd.DataFrame(
    similarity_scores,
    columns=["Cluster 0", "Cluster 1", "Cluster 2"],
    index=["Cluster 0", "Cluster 1", "Cluster 2"],
)
df_similarity_scores

### Concordance scores between clustering methods

In [ ]:
from sklearn.metrics import rand_score, adjusted_rand_score

In [ ]:
rand = rand_score(agglo_clustering, k_clustering)
rand

In [ ]:
adjusted_rand = adjusted_rand_score(agglo_clustering, k_clustering)
adjusted_rand

### Variations of the Agglomerative Clustering method

Now we try variations of the Agglomerative Clustering method by using different distance metrics that we want to compare.

In [ ]:
agglo_cosine = AgglomerativeClustering(n_clusters=3, metric="cosine", linkage="average")
agglo_eucli = AgglomerativeClustering(
    n_clusters=3, metric="euclidean", linkage="average"
)

In [ ]:
agglo_cosine_clusters = agglo_cosine.fit_predict(df_train_scaled).astype("int32")
agglo_eucli_clusters = agglo_eucli.fit_predict(df_train_scaled).astype("int32")

First, let's observe the size of each cluster, for each method.

In [ ]:
print(np.unique(agglo_cosine_clusters))
print(np.unique(agglo_eucli_clusters))

In [ ]:
# plot the two distributions of classes on each subplot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].set_title("Cosine distance")
axes[1].set_title("Euclidean distance")
# hist plot
sns.histplot(
    agglo_cosine_clusters, ax=axes[0], discrete=True, stat="percent", shrink=0.9
)
axes[0].xaxis.set_ticks([0, 1, 2], ["Cluster-0", "Cluster-1", "Cluster-2"])
sns.histplot(
    agglo_eucli_clusters, ax=axes[1], discrete=True, stat="percent", shrink=0.9
)
axes[1].xaxis.set_ticks([0, 1, 2], ["Cluster-0", "Cluster-1", "Cluster-2"])

Using the Euclidian distance with the agglomerative method yields a poor representation of the dataset, as all the samples are attributed to the same cluster.

Now, let's visualize the clusters generated using the cosine distance on the 2D-projection from part 1.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

sns.scatterplot(x=np_pca_projected[:, 0], y=np_pca_projected[:, 1], ax=axes[0], hue=agglo_cosine_clusters, palette="muted6")
axes[0].set_title("PCA")

sns.scatterplot(x=np_umap_projected[:, 0], y=np_umap_projected[:, 1], ax=axes[1], hue=agglo_cosine_clusters, palette="muted6")
axes[1].set_title("UMAP")

sns.scatterplot(x=np_tsne_projected[:, 0], y=np_tsne_projected[:, 1], ax=axes[2], hue=agglo_cosine_clusters, palette="muted6")
axes[2].set_title("t-SNE")

The clusters found using the cosine distance are very different from the ones found with the default parameters.

Let's quantify the similarity between these clusters.

In [ ]:
similarity_scores = np.zeros((3, 3))
for i in range(3):
    for j in range(3):
        similarity_score = jaccard_score(
            (agglo_cosine_clusters == i).astype("int32"),
            (agglo_eucli_clusters == j).astype("int32"),
        )
        similarity_scores[i, j] = similarity_score
        print(f"Similarity score for clusters {i} vs {j}: {similarity_score}")

In [ ]:
pd.DataFrame(
    similarity_scores,
    columns=["Cluster 0", "Cluster 1", "Cluster 2"],
    index=["Cluster 0", "Cluster 1", "Cluster 2"],
)

In [ ]:
rand = rand_score(agglo_cosine_clusters, agglo_eucli_clusters)
adjusted_rand = adjusted_rand_score(agglo_cosine_clusters, agglo_eucli_clusters)
print(f"Rand score: {rand}")
print(f"Adjusted rand score: {adjusted_rand}")

In [ ]:
len(agglo_eucli_clusters == 0)

All the samples are gathered in the same cluster using the euclidean distance.

## Regression

### Label pre-processing

In [ ]:
labels = pd.read_csv("data/gdsc_dr.csv", index_col=0, header=0)
labels.head()

In [ ]:
labels = labels.transpose()
labels.head()

In [ ]:
# merging training data on indexes
train = pd.merge(
    df_train_scaled, labels["doxorubicin"], left_index=True, right_index=True
)
train["doxorubicin"].isnull().sum()

In [ ]:
# droping missing values
train = train.dropna()
len(train)

In [ ]:
# scaling train labels
from sklearn.preprocessing import StandardScaler

label_scaler = StandardScaler()
train["doxorubicin"] = label_scaler.fit_transform(
    train["doxorubicin"].values.reshape(-1, 1)
).reshape(-1)

In [ ]:
# test data scaled using the trained scalertest
np_test_scaled = features_scaler.transform(df_test)
df_test_scaled = pd.DataFrame(
    np_test_scaled, index=df_test.index, columns=df_test.columns
)
test = pd.merge(
    df_test_scaled, labels["doxorubicin"], left_index=True, right_index=True
)
test["doxorubicin"].isnull().sum()

In [ ]:
test = test.dropna()
len(test)

In [ ]:
test["doxorubicin"] = label_scaler.transform(
    test["doxorubicin"].values.reshape(-1, 1)
).reshape(-1)

### Lasso regression

Scikit learn provides an implementation for the lasso regression that we will use.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

In [ ]:
Y = train["doxorubicin"]
X = train.drop("doxorubicin", axis=1)
Y_test = test["doxorubicin"]
X_test = test.drop("doxorubicin", axis=1)

In [ ]:
reg = Lasso(alpha=0.1, max_iter=10000)
reg.fit(X, Y)

In [ ]:
# evaluate on train set with Spearman correlation
Y_pred = reg.predict(X_test)
res = spearmanr(Y_test.values, Y_pred)
print(f"Spearman correlation on train set: {res.statistic}")

In [ ]:
print(f"Number of coefficients: {len(reg.coef_)}")
print(f"Number of non-zero coefficients (selected features): {np.sum(reg.coef_ != 0)}")

Now iterating over different values for $\alpha \in$ [0.01, 0.1, 0.3, 0.5, 0.9].

In [ ]:
nbr_non_zero = []
mse_scores = []
spearman_scores = []
alphas = [0.01, 0.1, 0.3, 0.5, 0.9]
for alpha in alphas:
    reg = Lasso(alpha=alpha, max_iter=10000)
    reg.fit(X, Y)
    mse_scores.append(mean_squared_error(Y_test, reg.predict(X_test)))
    spearman_scores.append(spearmanr(Y_test.values, reg.predict(X_test)).statistic)
    nbr_non_zero.append(np.sum(reg.coef_ != 0))

In [ ]:
spearman_scores

We observe that spearman correlation rank is not defined for higher $\alpha$ values because the regression output is close to a constant, as the number of selected features decreases to close to 0, so the coefficient diverges.

In [ ]:
# plot of number of non zero coefficients
sns.lineplot(x=alphas, y=nbr_non_zero, label="Number of selected features", marker="o")
plt.xlabel("alpha")
plt.ylabel("Number of non-zero coefficients")
plt.xticks(alphas)

### Nested Cross Validation

To compute the nested CV, we start anew from the unsplit and un scaled dataset, as the nested process involves splitting into test and train sets.

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import spearmanr


def spearman_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    res = spearmanr(y_true, y_pred).statistic
    if res is np.nan:
        return 1000
    return res

In [ ]:
# unpreprocessed features and labels
labels = pd.read_csv("data/gdsc_dr.csv", index_col=0, header=0).transpose()[
    "doxorubicin"
]
features = pd.read_csv("data/gdsc_expr_postCB.csv", index_col=0, header=0).transpose()
df_full = pd.merge(features, labels, left_index=True, right_index=True)
print(f"Dataset shape : {df_full.values.shape}")
print(f"Number of missing values : {df_full.isnull().sum().sum()}")

In [ ]:
df_full = df_full.dropna()

In [ ]:
p_grid = {"reg__alpha": [0.01, 0.1, 0.3, 0.5, 0.9]}
pipe = Pipeline(steps=[("scaler", StandardScaler()), ("reg", Lasso(max_iter=10000))])

In [ ]:
Y = df_full["doxorubicin"]
X = df_full.drop("doxorubicin", axis=1)

inner_cv = KFold(n_splits=3, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=4, shuffle=True, random_state=0)

clf = GridSearchCV(estimator=pipe, param_grid=p_grid, cv=inner_cv)
nested_scores = cross_validate(
    clf,
    X=X,
    y=Y,
    cv=outer_cv,
    scoring={"mse": make_scorer(mean_squared_error), "spearman": spearman_scorer},
    return_train_score=True,
    return_estimator=True,
    n_jobs=-1,
    verbose=1,
)

In [ ]:
import joblib

joblib.dump(nested_scores, "nested_scores.pkl")

In [ ]:
import joblib

nested_scores = joblib.load("nested_scores.pkl")

In [ ]:
nested_scores

In [ ]:
avg_mse = np.mean(nested_scores["test_mse"])
avg_spearman = np.mean(nested_scores["test_spearman"])
print(f"Average MSE: {avg_mse:.3f}")
print(f"Average Spearman correlation: {avg_spearman:.3f}")

In [ ]:
avg_mse = np.mean(nested_scores["train_mse"])
avg_spearman = np.mean(nested_scores["train_spearman"])
print(f"Average MSE: {avg_mse:.3f}")
print(f"Average Spearman correlation: {avg_spearman:.3f}")

In [ ]:
best_alphas = []
for est in nested_scores["estimator"]:
    best_alphas.append(est.best_params_["reg__alpha"])
print(best_alphas)